# 天池第三届运维大赛
## https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.18.164e12bbVlwhLI&postId=341392

# 1.引入包

In [1]:
import os
import nltk
# nltk.download('punkt')
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier

# 2. 读取数据

In [2]:
# 读取sel日志，排序
sel_data = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_sel_log_dataset.csv')
sel_data.sort_values(by=['sn', 'time'], inplace=True)
sel_data.reset_index(drop=True, inplace=True)

In [3]:
sel_data.head()

,sn,time,msg,server_model
0,SERVER_10001,2020-05-01 08:54:43,Processor CPU1_Status | IERR | Asserted,SM57
1,SERVER_10001,2020-05-01 08:54:43,Processor CPU0_Status | IERR | Asserted,SM57
2,SERVER_10001,2020-05-01 08:55:03,Management Subsys Health System_Health | Sens...,SM57
3,SERVER_10001,2020-05-01 08:59:48,Processor CPU0_Status | IERR | Deasserted,SM57
4,SERVER_10001,2020-05-01 08:59:48,Processor CPU1_Status | IERR | Deasserted,SM57


# 3. 分词

In [5]:
# 取出每台服务器的最后十条日志
sn_list = sel_data['sn'].drop_duplicates(keep='first').to_list() # drop_duplicates一种是去除完全重复的行数据，另一种是去除某几列重复的行数据
tail_msg_list = ['.'.join(sel_data[sel_data['sn']==i]['msg'].tail(10).to_list()) for i in sn_list]
tokenized_sent = [word_tokenize(s.lower()) for s in tail_msg_list]

In [15]:
print(tail_msg_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
print(tokenized_sent)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 4.训练embbeding模型（Doc2Vec）

In [7]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
model = Doc2Vec(tagged_data, vector_size = 10, window = 2, min_count = 1, epochs = 10)

# 5. 构建树模型的训练集

In [8]:
label = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_train_label_dataset.csv')
label.sort_values(by=['sn', 'fault_time'], inplace=True)
label.reset_index(drop=True, inplace=True)
train_data = []
for i, row in label.iterrows():
    train_data.append(model.infer_vector(word_tokenize('.'.join(sel_data[(sel_data['sn']==row['sn'])&(sel_data['time']<=row['fault_time'])].tail(10)['msg']).lower())))
train_feature = np.array(train_data)
train_label = label['label'].values

In [9]:
print(train_feature)

[[-0.08868866  0.07740986 -0.19132008 ... -0.01196239  0.08234586
  -0.00191107]
 [ 0.09758941 -0.03947885 -0.19215561 ...  0.00401722 -0.00112704
  -0.01598878]
 [ 0.04918076  0.00552294 -0.07873591 ... -0.02197392 -0.09757466
  -0.10528761]
 ...
 [-0.00514735  0.09542491 -0.10804312 ... -0.05724438  0.05273888
   0.00293469]
 [ 0.04705299  0.03412386 -0.1034546  ... -0.10106593 -0.14732324
  -0.11483219]
 [-0.01729597 -0.12000942 -0.22586234 ...  0.10263926  0.09419995
  -0.15045395]]


In [14]:
print(train_data)

[array([-0.08868866,  0.07740986, -0.19132008, -0.03030667,  0.36378354,
       -0.22469306,  0.3628645 , -0.01196239,  0.08234586, -0.00191107],
      dtype=float32), array([ 0.09758941, -0.03947885, -0.19215561, -0.01180615,  0.09890792,
       -0.11049386,  0.28299052,  0.00401722, -0.00112704, -0.01598878],
      dtype=float32), array([ 0.04918076,  0.00552294, -0.07873591, -0.11657974, -0.08265904,
       -0.08306839,  0.23265041, -0.02197392, -0.09757466, -0.10528761],
      dtype=float32), array([ 0.18239312,  0.02434428, -0.0730057 , -0.15092435, -0.03965772,
       -0.24359967,  0.42416018, -0.14713833, -0.19459641, -0.0198621 ],
      dtype=float32), array([ 0.16894126,  0.16592811, -0.1254576 , -0.11383551,  0.26977938,
       -0.0434667 ,  0.4675093 , -0.19166102, -0.17593543, -0.0216061 ],
      dtype=float32), array([-0.02351252,  0.17586845, -0.29289004, -0.00892994,  0.46450526,
       -0.05737843,  0.52190155,  0.08950154, -0.2684378 ,  0.16886793],
      dtype=float32

# 6. 训练随机森林

In [10]:
rf = RandomForestClassifier(oob_score=True)
rf.fit(train_feature, train_label)

RandomForestClassifier(oob_score=True)

# 7. 构建测试集

In [12]:
submit = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_submit_dataset_a.csv')
submit.sort_values(by=['sn', 'fault_time'], inplace=True)
submit.reset_index(drop=True, inplace=True)
test_data = []
for i, row in submit.iterrows():
    test_data.append(model.infer_vector(word_tokenize('. '.join(sel_data[(sel_data['sn']==row['sn'])&(sel_data['time']<=row['fault_time'])].tail(10)['msg']).lower())))
test_feature = np.array(test_data)

# 8. 预测并保存结果

In [13]:
test_label = rf.predict(test_feature)
submit['label'] = test_label
submit.to_csv('./preliminary_pred_df_Emdedding.csv', index=0)

线上得分为0.1